In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from torch import nn

In [2]:

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-0.5B", trust_remote_code=True)


In [3]:
# Load the model with CPU (avoiding the `device_map` parameter)
model = AutoModelForCausalLM.from_pretrained(
    "Qwen/Qwen2.5-0.5B",
    trust_remote_code=True,
    torch_dtype=torch.float16,
    output_hidden_states=True  # Enable hidden state outputs
).to("cpu")


/Users/amirali/Desktop/Final-Year-Project/.venv/lib/python3.12/site-packages/transformers/generation/configuration_utils.py:818: UserWarning: `return_dict_in_generate` is NOT set to `True`, but `output_hidden_states` is. When `return_dict_in_generate` is not `True`, `output_hidden_states` is ignored.
  warnings.warn(


## Extracts Layer Representations

In [4]:
# Custom forward pass to extract hidden states
def get_layer_representations(input_text):
    tokens = tokenizer(input_text, return_tensors="pt", truncation=True, padding=True).to("cpu")
    outputs = model(**tokens)
    hidden_states = outputs.hidden_states  # Tuple of hidden states for all layers
    return hidden_states

In [5]:
# Ask a question as input
input_text = "Hey mate how are you today, you are a great assistant... In the sentence you just processed what emotion was expressed ?"

In [6]:
# Tokenize the input
encoded_input = tokenizer(
    input_text, 
    return_tensors="pt", 
    padding=True, 
    truncation=True
).to("cpu")

In [7]:
# Generate a response
output_ids = model.generate(
    input_ids=encoded_input["input_ids"].to("cpu"),
    attention_mask=encoded_input["attention_mask"].to("cpu"),
    max_new_tokens=500,  # Avoid using `max_length` to prevent conflicts
    num_beams=2,
    num_return_sequences=2
)


Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


In [8]:

# Decode the output
response = tokenizer.decode(output_ids[0], skip_special_tokens=True)

print(response)


Hey mate how are you today, you are a great assistant... In the sentence you just processed what emotion was expressed ? The emotion expressed in the sentence "Hey mate how are you today, you are a great assistant... In the sentence "Hey mate how are you today, you are a great assistant...", the emotion expressed is a positive one. The use of the word "great" indicates that the speaker is pleased or happy with their current situation.


In [9]:
class ClassifierProbe(nn.Module):
    def __init__(self, input_dim, num_classes):
        super(ClassifierProbe, self).__init__()
        self.fc = nn.Linear(input_dim, num_classes)
    
    def forward(self, x):
        return self.fc(x)


## Prepare GoEmotion Data

In [10]:
import pandas as pd 
import re
import emoji

In [11]:
GoEmotion_train= pd.read_csv('Go_Emotion_Google/go_emotions_train.csv')
GoEmotion_test = pd.read_csv('Go_Emotion_Google/go_emotions_test.csv')
GoEmotion_val = pd.read_csv('Go_Emotion_Google/go_emotions_validation.csv')

df_train = pd.DataFrame(GoEmotion_train)
df_test = pd.DataFrame(GoEmotion_test)
df_validation = pd.DataFrame(GoEmotion_val)


df_train = df_train[:5000]
df_test = df_test[:5000]
df_validation = df_validation[:5000]

print("\tTrain : \n", df_train["text"].head(10), "\n\n\tTest : \n", df_test["text"].head(10), "\n\n\tValidation : \n", df_validation["text"].head(10))


	Train : 
 0    My favourite food is anything I didn't have to...
1    Now if he does off himself, everyone will thin...
2                       WHY THE FUCK IS BAYLESS ISOING
3                          To make her feel threatened
4                               Dirty Southern Wankers
5    OmG pEyToN iSn'T gOoD eNoUgH tO hElP uS iN tHe...
6    Yes I heard abt the f bombs! That has to be wh...
7    We need more boards and to create a bit more s...
8    Damn youtube and outrage drama is super lucrat...
9    It might be linked to the trust factor of your...
Name: text, dtype: object 

	Test : 
 0    I’m really sorry about your situation :( Altho...
1      It's wonderful because it's awful. At not with.
2    Kings fan here, good luck to you guys! Will be...
3    I didn't know that, thank you for teaching me ...
4    They got bored from haunting earth for thousan...
5    Thank you for asking questions and recognizing...
6                                       You’re welcome
7               

In [12]:
def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()
    # Remove unwanted punctuations
    
    # Handle specific punctuations and emoticons
    emoticon_dict = {
        ":)": "happy",
        ":))": "happy",
        ":-)": "happy",
        ":-))": "happy",
        ":(": "sad",
        ":((": "sad",
        ":-((": "sad",
        ":-((": "sad",
        ":/": "confusion", 
        "://": "confusion", 
        ":-/": "confusion",
        ":-//": "confusion",
        ":\\": "confusion",
        ":-\\": "confusion",
        ":|": "neutral",
        ":-|": "neutral",
        "XD": "laugh",
        ":D": "laugh",
        ":-D": "laugh"
    }
    
    # Replace emoticons with words
    for emoticon, replacement in emoticon_dict.items():
        text = text.replace(emoticon, replacement)
    
    # Handle emojis
    text = emoji.demojize(text)
    
    # Remove non-alphanumeric characters except spaces (next line removes unnecessary spaces)
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    
    return text


# Apply preprocessing to the text column
df_train['cleaned_text'] = df_train['text'].apply(preprocess_text)
df_test['cleaned_text'] = df_test['text'].apply(preprocess_text)
df_validation['cleaned_text'] = df_validation['text'].apply(preprocess_text)

In [13]:
def string_to_list(label_input):
    """
    Convert string representation of an array into a list of integers.
    
    Args:
    label_str (str): String representation of an array, e.g., '[6 7]'.
    
    Returns:
    list: A list of integers.
    """
    if isinstance(label_input, list):
        # If it's already a list, return it as-is
        return [int(x) for x in label_input]
    elif isinstance(label_input, str):
        # If it's a string, parse it
        return [int(x) for x in label_input.strip('[]').replace(',', '').split()]
    else:
        raise ValueError("label_input must be a string or a list.")

In [14]:
# The index of each emotion refers to the value stored for each emotion as label, those can be translated by these explanations 
emotions = ['admiration', 'amusement', 'anger', 'annoyance', 'approval', 'caring', 'confusion', 'curiosity', 'desire',
            'disappointment', 'disapproval', 'disgust', 'embarrassment', 'excitement', 'fear', 'gratitude', 'grief',
            'joy', 'love', 'nervousness','optimism', 'pride', 'realization', 'relief', 'remorse', 'sadness', 'surprise', 'neutral']

positive = {"admiration", "amusement", "approval", "caring", "desire", "excitement",
            "gratitude", "joy", "love", "optimism", "pride", "relief"}

ambiguous = {"confusion", "curiosity", "surprise", "realization", "neutral"}

negative = {"anger", "annoyance", "disappointment", "disapproval", "disgust",
            "embarrassment", "fear", "grief", "nervousness", "remorse", "sadness"}

In [15]:
All_DF = [df_train, df_test, df_validation]

for df in All_DF:
    df['emotions'] = None

    for i in range(len(df['text'])):

        indexes = df['labels'][i]
        parsed_indexes = list(string_to_list(indexes))
        
        df.at[i, 'labels'] = str(parsed_indexes)
        
        emotion_texts = []
        for index in parsed_indexes:
            emotion_texts.append(emotions[index])
        
        df.at[i, 'emotions'] = emotion_texts
        

In [16]:
train_texts = df_train["cleaned_text"]
train_labels = df_train["labels"]
val_texts = df_validation["cleaned_text"]
val_labels = df_validation["labels"]

In [17]:
from torch.utils.data import DataLoader, Dataset
import torch

class GoEmotionDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        tokens = self.tokenizer(
            self.texts[idx],
            truncation=True,
            padding="max_length",
            max_length=self.max_length,
            return_tensors="pt"
        )
        tokens = {k: v.squeeze(0) for k, v in tokens.items()}  # Squeeze batch dimension
        
        # Convert labels from string lists to numeric indices
        label_str = self.labels[idx]  # This is the label string that needs parsing
        
        # Ensure label_str is a list of integers (emotions indices)
        try:
            parsed_labels = string_to_list(label_str)
        except ValueError as e:
            print(f"Error parsing label: {label_str}")
            raise e
        
        # Convert to a tensor (using multi-labels, it's often common to use float32 for multi-hot)
        label_tensor = torch.tensor(parsed_labels, dtype=torch.float32)
        
        return tokens, label_tensor



In [18]:
def custom_collate_fn(batch):
    token_keys = batch[0][0].keys()
    collated_tokens = {key: torch.stack([item[0][key] for item in batch]) for key in token_keys}
    
    # Find the maximum label length in the batch
    max_label_len = max([len(item[1]) for item in batch])
    
    # Pad labels to ensure they are all the same size
    padded_labels = []
    for item in batch:
        label_tensor = item[1]
        padding = max_label_len - len(label_tensor)
        padded_labels.append(torch.cat([label_tensor, torch.zeros(padding, dtype=torch.float32)]))
    
    # Stack the padded labels
    labels = torch.stack(padded_labels)
    
    return collated_tokens, labels


In [19]:
# Define train and validation datasets
train_dataset = GoEmotionDataset(train_texts.tolist(), train_labels.tolist(), tokenizer)
val_dataset = GoEmotionDataset(val_texts.tolist(), val_labels.tolist(), tokenizer)

# Define DataLoaders with custom collate function
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, collate_fn=custom_collate_fn)
val_loader = DataLoader(val_dataset, batch_size=16, collate_fn=custom_collate_fn)

In [20]:
from torch import nn
import torch.optim as optim
from sklearn.model_selection import train_test_split

In [21]:
# Get hidden state sizes for initializing probes
hidden_state_sizes = [
    layer.shape[-1] for layer in model(**tokenizer("example text", return_tensors="pt")).hidden_states
]
num_classes = 28  # Assuming labels are integers starting from 0
probes = [ClassifierProbe(size, num_classes).to("cpu") for size in hidden_state_sizes]

In [22]:
# Training loop
loss_fn = nn.CrossEntropyLoss()
optimizers = [optim.Adam(probe.parameters(), lr=1e-4) for probe in probes]

In [23]:
All_DF = [df_train, df_test, df_validation]

for df in All_DF:
    for i in range(len(df['text'])):

        indexes = df['labels'][i]
        parsed_indexes = list(string_to_list(indexes))
        
        df.at[i, 'labels'] = str(parsed_indexes)
        
        

In [24]:
df_train['emotions']

0                  [neutral]
1                  [neutral]
2                    [anger]
3                     [fear]
4                [annoyance]
                ...         
4995               [neutral]
4996            [admiration]
4997                  [fear]
4998             [gratitude]
4999    [amusement, sadness]
Name: emotions, Length: 5000, dtype: object

In [25]:
for epoch in range(3):  # Example with 3 epochs
    model.eval()  # Ensure the model stays in evaluation mode
    for tokens, labels in train_loader:
        # Ensure correct shapes
        tokens = {k: v.squeeze(0).to("cpu") if v.dim() == 3 else v.to("cpu") for k, v in tokens.items()}
        labels = labels.to("cpu")
        
        # Debug: Inspect token shapes
        print(f"Input IDs shape: {tokens['input_ids'].shape}")
        print(f"Attention mask shape: {tokens['attention_mask'].shape}")

        try:
            outputs = model(**tokens, output_hidden_states=True)
        except RuntimeError as e:
            print("Error during model forward pass:", e)
            continue

        hidden_states = outputs.hidden_states

        # Train each probe on its corresponding layer
        # Process each label
        for i, (probe, optimizer) in enumerate(zip(probes, optimizers)):
            optimizer.zero_grad()
            
            # Convert labels to indices
            label_str = df_train['labels'][i]  # Assuming this is the label in string format
            label_list = string_to_list(label_str)
            
            print(f"label_list: {label_list}")
            print(f"emotions: {emotions}")

                        
            # Create reverse mapping
            emotion_to_index = {emotion: idx for idx, emotion in enumerate(emotions)}

            # Convert labels to indices using the dictionary
            try:
                label_indices = [emotion_to_index[emotion] for emotion in label_list]
            except KeyError as e:
                print(f"Unknown emotion in label_list: {label_list}. Error: {e}")
                continue

            # Detach the hidden states for the current layer
            layer_rep = hidden_states[i].detach()  # Detach to avoid backprop through the main model
            
            logits = probe(layer_rep)
            loss = loss_fn(logits, label_indices)  # Assuming label_indices are properly formatted
            loss.backward()
            optimizer.step()

        print(f"Epoch {epoch + 1} complete.")



Input IDs shape: torch.Size([16, 128])
Attention mask shape: torch.Size([16, 128])
label_list: [27]
emotions: ['admiration', 'amusement', 'anger', 'annoyance', 'approval', 'caring', 'confusion', 'curiosity', 'desire', 'disappointment', 'disapproval', 'disgust', 'embarrassment', 'excitement', 'fear', 'gratitude', 'grief', 'joy', 'love', 'nervousness', 'optimism', 'pride', 'realization', 'relief', 'remorse', 'sadness', 'surprise', 'neutral']
Unknown emotion in label_list: [27]. Error: 27
label_list: [27]
emotions: ['admiration', 'amusement', 'anger', 'annoyance', 'approval', 'caring', 'confusion', 'curiosity', 'desire', 'disappointment', 'disapproval', 'disgust', 'embarrassment', 'excitement', 'fear', 'gratitude', 'grief', 'joy', 'love', 'nervousness', 'optimism', 'pride', 'realization', 'relief', 'remorse', 'sadness', 'surprise', 'neutral']
Unknown emotion in label_list: [27]. Error: 27
label_list: [2]
emotions: ['admiration', 'amusement', 'anger', 'annoyance', 'approval', 'caring', 'co

In [ ]:

# Validation
print("Starting validation...")
model.eval()
with torch.no_grad():
    for tokens, labels in val_loader:
        tokens = {k: v.squeeze(0).to("cpu") for k, v in tokens.items()}
        labels = labels.to("cpu")
        outputs = model(**tokens, output_hidden_states=True)
        hidden_states = outputs.hidden_states

        for i, probe in enumerate(probes):
            layer_rep = hidden_states[i]
            logits = probe(layer_rep)
            loss = loss_fn(logits, labels)
            print(f"Validation Loss for Layer {i}: {loss.item()}")

TypeError: new(): invalid data type 'str'